In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import numpy as np
import torch
from typing import Tuple
import math
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import trimesh
import open3d as o3d

In [ ]:
### Comment/Uncomment the following line to switch between static and interactive plots
# matplotlib.use('Qt5Agg')
# %matplotlib widget

In [ ]:
# bunny_mesh = trimesh.load('bun_zipper_res4.ply')
# assert isinstance(bunny_mesh, trimesh.Trimesh), "Loaded object should be a Trimesh"
# rotation_matrix = trimesh.transformations.rotation_matrix(
#     np.radians(90), [1, 0, 0])
# bunny_mesh.apply_transform(rotation_matrix)

# vertices = bunny_mesh.vertices
# faces = bunny_mesh.faces

# N = 2000 # number of points to sample

# # Use bunny_mesh
# mesh = o3d.geometry.TriangleMesh()
# mesh.vertices = o3d.utility.Vector3dVector(vertices)
# mesh.triangles = o3d.utility.Vector3iVector(faces)
# mesh.compute_vertex_normals()

# pcd = mesh.sample_points_poisson_disk(N)
# pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),
#           center=pcd.get_center()) # fit to unit cube
# pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
# o3d.visualization.draw_geometries([pcd]) # type: ignore

# print('octree division')
# octree = o3d.geometry.Octree(max_depth=4)
# octree.convert_from_point_cloud(pcd, size_expand=0.01)
# o3d.visualization.draw_geometries([octree]) # type: ignore

- implicity calculate gradient instead of explicity store each position's IOR. The wavefront propagation stage doesn't need grid and the position of wavefront points doesn't need to be inside the grid.

-  try to test on random position random direction. 

- store results of propagation to generate irradiance in grid (do not need points afterwards)

## Initialize

In [ ]:
num_x, num_y, num_z = 50, 50, 50
ior_field = torch.ones((num_x, num_y, num_z), dtype=torch.float16, device=device)
absorb_coeff = torch.zeros((num_x, num_y, num_z), dtype=torch.float16, device=device)
scatter_coeff = torch.zeros((num_x, num_y, num_z), dtype=torch.float16, device=device)

##############  Set Parameters      ##############
materials = {
    'sphere': {'ior': 1.5, 'absorb': 0.001, 'scatter': 0.001},
    'wall': {'ior': 1.7, 'absorb': 0.5, 'scatter': 0.5}
}

spheres = [
    {'radius': 12, 'offset': (10, -10, 10)},
    {'radius': 10, 'offset': (-10, -10, -10)}
]

wall_thickness = 3

##############  Initialize scene    ##############
# [Sphere] Store sphere in grid
def set_sphere_properties(center, radius, material):
    x, y, z = torch.meshgrid(torch.arange(num_x), torch.arange(num_y), torch.arange(num_z), indexing='ij')
    sphere_mask = ((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2 <= radius**2).to(device)
    ior_field[sphere_mask] = material['ior']
    absorb_coeff[sphere_mask] = material['absorb']
    scatter_coeff[sphere_mask] = material['scatter']

sphere_center_base = (num_x//2, num_y//2, num_z//2)
for sphere in spheres:
    sphere_center = (
        sphere_center_base[0] +  sphere['offset'][0],
        sphere_center_base[1] +  sphere['offset'][1],
        sphere_center_base[2] +  sphere['offset'][2]
    )
    set_sphere_properties(sphere_center, sphere['radius'], materials['sphere'])

# [Walls] Set properties for walls
wall_indices = [
    (slice(None), slice(None), slice(-wall_thickness, None)),  # ceiling
    (slice(None), slice(None), slice(None, wall_thickness)),   # floor
    (slice(None), slice(wall_thickness), slice(None)),         # front
    (slice(None), slice(-wall_thickness, None), slice(None)),  # back
    (slice(wall_thickness), slice(None), slice(None)),         # left
    (slice(-wall_thickness, None), slice(None), slice(None))   # right
]

for indices in wall_indices:
    ior_field[indices] = materials['wall']['ior']
    absorb_coeff[indices] = materials['wall']['absorb']
    scatter_coeff[indices] = materials['wall']['scatter']

##############  Initialize wavefront positions and directions   ############
def spherical_to_cartesian(theta, phi):
    r = 1
    theta_rad = math.radians(theta)
    phi_rad = math.radians(phi)
    x = r * math.sin(theta_rad) * math.cos(phi_rad)
    y = r * math.sin(theta_rad) * math.sin(phi_rad)
    z = r * math.cos(theta_rad)
    return x, y, z


theta_step = 0.5
phi_step = 0.5
theta_angles = np.arange(90, 180, theta_step)
phi_angles = np.arange(0, 360, phi_step)
initial_wavefront_directions = torch.tensor([spherical_to_cartesian(theta, phi) for theta in theta_angles for phi in phi_angles], dtype=torch.float16, device='cuda')
initial_wavefront_positions = torch.tensor([(wall_thickness+2, wall_thickness+2, num_z - (wall_thickness + 3))], dtype=torch.float16, device='cuda')

In [ ]:
def show_ior_field(ior_field, initial_wavefront_positions, initial_wavefront_directions):    
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_box_aspect([1,1,1])
    ax.set_xlim(0, num_x)
    ax.set_ylim(0, num_y)
    ax.set_zlim(0, num_z)
    initial_pos_np = initial_wavefront_positions.cpu().numpy()
    initial_dir_np = initial_wavefront_directions.cpu().numpy()

    subsample_indices = np.arange(0, initial_wavefront_positions.shape[0], 2)  # Subsampling every 2 data point
    ax.quiver(initial_pos_np[subsample_indices, 0], initial_pos_np[subsample_indices, 1], initial_pos_np[subsample_indices, 2],
                initial_dir_np[subsample_indices, 0], initial_dir_np[subsample_indices, 1], initial_dir_np[subsample_indices, 2], color='g')
    
    # IOR field
    ior_field = ior_field.cpu().numpy()
    x_points, y_points, z_points = np.where(ior_field == materials['sphere']['ior'])
    ax.scatter(x_points, y_points, z_points, color='blue', alpha=0.02)
    x_points, y_points, z_points = np.where(ior_field == materials['wall']['ior'])
    ax.scatter(x_points, y_points, z_points, color='red', alpha=0.02)
    ax.set_title("Glass Sphere (blue color) inside a room with walls (red color) with a point light source at the top left corner of the room (green arrows)")
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    plt.show()

show_ior_field(ior_field, initial_wavefront_positions, initial_wavefront_directions)

In [ ]:
def visualize_initial_directions():
    sample_rate = 65
    positions_np = initial_wavefront_positions.cpu().numpy()
    directions_np = initial_wavefront_directions.cpu().numpy()
    sampled_positions = positions_np[::sample_rate, :]
    sampled_directions = directions_np[::sample_rate, :]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.quiver(sampled_positions[:, 0], sampled_positions[:, 1], sampled_positions[:, 2],
            sampled_directions[:, 0], sampled_directions[:, 1], sampled_directions[:, 2], length=1.5, normalize=True)
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    plt.show()

def compute_3d_gradients_torch(IOR):
    grad_x, grad_y, grad_z = torch.gradient(IOR)
    return grad_x, grad_y, grad_z

def visualize_gredients(convex_grad_xyz_torch):
    grad_x = convex_grad_xyz_torch[0].cpu().numpy()
    grad_y = convex_grad_xyz_torch[1].cpu().numpy()
    grad_z = convex_grad_xyz_torch[2].cpu().numpy()
    plt.figure(figsize=(8, 3))
    plt.subplot(131)
    plt.imshow(grad_x[:, :, num_z // 2], cmap='gray')  # Z axis cut-through
    plt.title('Gradient along x-axis')
    plt.xlabel('x-axis')
    plt.ylabel('y-axis')
    plt.subplot(132)
    plt.imshow(grad_y[num_y // 2, :, :], cmap='gray')  # Y axis cut-through
    plt.title('Gradient along y-axis')
    plt.xlabel('x-axis')
    plt.ylabel('z-axis')
    plt.subplot(133)
    plt.imshow(grad_z[:, num_x // 2, :], cmap='gray')  # X axis cut-through
    plt.title('Gradient along z-axis')
    plt.xlabel('y-axis')
    plt.ylabel('z-axis')
    plt.tight_layout()
    plt.show()

convex_grad_xyz_torch = compute_3d_gradients_torch(ior_field)
# visualize_gredients(convex_grad_xyz_torch)

## Wavefront propagation

In [ ]:
@torch.jit.script
def update_wavefront(pos: torch.Tensor, dir: torch.Tensor, grad_xyz: Tuple[torch.Tensor, torch.Tensor, torch.Tensor], IOR: torch.Tensor, delta_t: float) -> Tuple[torch.Tensor, torch.Tensor]:

    ## 1. When the data size is small, update the whole wavefront at once
    indices = pos.int()
    # Obtain the refractive index at the current position
    n = IOR[indices[:, 0].clamp(0, IOR.shape[0] - 1),
            indices[:, 1].clamp(0, IOR.shape[1] - 1),
            indices[:, 2].clamp(0, IOR.shape[2] - 1)]

    # Calculate the new position and direction of the wavefront
    new_pos = pos + delta_t * dir / (n**2).unsqueeze(1)

    # Create a mask to ensure that the wavefront stays within the boundaries of the IOR field
    mask = (new_pos[:, 0] >= 0) & (new_pos[:, 0] < IOR.shape[0]) & \
           (new_pos[:, 1] >= 0) & (new_pos[:, 1] < IOR.shape[1]) & \
           (new_pos[:, 2] >= 0) & (new_pos[:, 2] < IOR.shape[2])
    mask = mask.unsqueeze(1)  # Add a dimension to match the shape of new_pos

    # only update the positions that are within the IOR boundaries
    new_pos = torch.where(mask, new_pos, pos)
    new_dir = torch.where(mask, dir + delta_t * torch.stack([grad_xyz[0][indices[:, 0], indices[:, 1], indices[:, 2]],  grad_xyz[1][indices[:, 0], indices[:, 1], indices[:, 2]], grad_xyz[2][indices[:, 0], indices[:, 1], indices[:, 2]]], dim=1) / n.unsqueeze(1), dir)

    ## 2. TODO: When the data size is large, update the wavefront in chunks with adaptive IOR field and tree data structure
    # octree = o3d.geometry.Octree(max_depth=4)

    return new_pos, new_dir

@torch.jit.script
def simulate_wavefront_propagation(cur_IOR: torch.Tensor, grad_xyz: Tuple[torch.Tensor, torch.Tensor, torch.Tensor], initial_wavefront_pos: torch.Tensor, 
                                   initial_wavefront_dir: torch.Tensor, num_steps: int = 100, delta_t: float = 1.0):
    wavefront_pos_list = [initial_wavefront_pos]
    wavefront_dir_list = [initial_wavefront_dir]
    # for_i trick
    for _ in range(num_steps):
        new_positions, new_directions = update_wavefront(wavefront_pos_list[-1], wavefront_dir_list[-1], grad_xyz, cur_IOR, delta_t)
        wavefront_pos_list.append(new_positions)
        wavefront_dir_list.append(new_directions)
    return wavefront_pos_list, wavefront_dir_list

test_num_steps = 200
test_delta_t = 1.0 * num_x / 100
wavefront_positions_list, wavefront_directions_list = simulate_wavefront_propagation(ior_field, convex_grad_xyz_torch, initial_wavefront_positions, initial_wavefront_directions, num_steps=test_num_steps, delta_t=test_delta_t)

In [ ]:
def show_propagation_results(convex_pos_res, convex_dir_res, IOR, num_show_images, fig_name):
    num_cols = min(num_show_images, 3)
    num_rows = (num_show_images + num_cols - 1) // num_cols
    num_steps = len(convex_pos_res) - 1

    show_indices = [i for i in range(0, num_steps+1, max(num_steps // (num_show_images-1), 1))]
    show_indices.append(num_steps)
    if len(show_indices) > num_show_images:
        show_indices = show_indices[:num_show_images]

    fig = plt.figure(figsize=(6 * num_cols, 6 * num_rows))
    
    for i, index in enumerate(show_indices):
        ax = fig.add_subplot(num_rows, num_cols, i + 1, projection='3d')
        ax.set_box_aspect([1,1,1])
        ax.set_xlim(0, IOR.shape[0])
        ax.set_ylim(0, IOR.shape[1])
        ax.set_zlim(0, IOR.shape[2])
        ax.set_xlabel('X Axis')
        ax.set_ylabel('Y Axis')
        ax.set_zlabel('Z Axis')
        ax.set_title(f"Step {index}")
        
        # Visualize 3D IOR field
        IOR_np = IOR.cpu().numpy() if IOR.is_cuda else IOR.numpy()
        x_points, y_points, z_points = np.where(IOR_np == materials['sphere']['ior'])
        ax.scatter(x_points, y_points, z_points, color='blue', alpha=0.04)

        # Visualize Wavefront
        wavefront_positions = convex_pos_res[index].cpu().numpy() if convex_pos_res[index].is_cuda else convex_pos_res[index].numpy()
        wavefront_directions = convex_dir_res[index].cpu().numpy() if convex_dir_res[index].is_cuda else convex_dir_res[index].numpy()
        subsample_indices = np.arange(0, wavefront_positions.shape[0], 2)  # Subsampling every 2 data point
        ax.quiver(wavefront_positions[subsample_indices, 0], wavefront_positions[subsample_indices, 1], wavefront_positions[subsample_indices, 2], 
                  wavefront_directions[subsample_indices, 0], wavefront_directions[subsample_indices, 1], wavefront_directions[subsample_indices, 2],
                  length=1, color='green', normalize=True, linewidths=0.7, arrow_length_ratio=0.4, alpha=0.4)
        
    fig.tight_layout()
    fig.suptitle(fig_name)
    plt.show()

num_show_images = 6
show_propagation_results(wavefront_positions_list, wavefront_directions_list, ior_field, num_show_images, "Wavefront Propagation of the glass sphere inside a room with walls")

## Radiometric (Irradiance or Radiance) Computation

In [ ]:
def update_irradiance(pos, irradiance, absorb_coeff, scatter_coeff, delta_t):
    # Retrieve the absorption and scattering coefficients at the current position
    indices = pos.int()
    absorption = absorb_coeff[indices[:, 0], indices[:, 1], indices[:, 2]]
    scattering = scatter_coeff[indices[:, 0], indices[:, 1], indices[:, 2]]

    # Calculate the decay factor based on the Beer-Lambert law
    # TODO: Not completely correspond to the original paper yet
    decay_factor = torch.exp(-absorption * delta_t) * (1 - scattering)
    new_irradiance = irradiance * decay_factor
    return new_irradiance

initial_irradiance = torch.ones_like(initial_wavefront_positions[:, 0])  # Assume the initial irradiance is 1
irradiance_list = [initial_irradiance]

# Simulate the process of irradiance decay
for i in range(1, len(wavefront_positions_list)):
    new_irradiance = update_irradiance(
        wavefront_positions_list[i],
        irradiance_list[-1],
        absorb_coeff,
        scatter_coeff,
        test_delta_t
    )
    irradiance_list.append(new_irradiance)

In [ ]:
def update_radiance(pos, dir, irradiance, absorb_coeff, scatter_coeff, delta_t, camera_direction):
    # Retrieve the absorption and scattering coefficients at the current position
    indices = pos.int()
    absorption = absorb_coeff[indices[:, 0], indices[:, 1], indices[:, 2]]
    scattering = scatter_coeff[indices[:, 0], indices[:, 1], indices[:, 2]]

    # Calculate the decay factor based on the Beer-Lambert law
    decay_factor = torch.exp(-absorption * delta_t) * (1 - scattering)
    
    # Calculate the directional factor based on the angle between the wavefront direction and the camera direction
    # Normalize the wavefront direction and camera direction
    dir_normalized = dir / torch.norm(dir, dim=1, keepdim=True)
    camera_direction_normalized = camera_direction / torch.norm(camera_direction)
    
    # Calculate the dot product between the normalized wavefront direction and the camera direction
    directional_factor = torch.sum(dir_normalized * camera_direction_normalized, dim=1)
    directional_factor = torch.clamp(directional_factor, min=0)  # Consider only the component in the direction of the camera
    
    # Calculate the new radiance
    new_radiance = irradiance * decay_factor * directional_factor
    return new_radiance

camera_position = torch.tensor([num_x/2, num_y/2, num_z + 10], dtype=torch.float16, device=device)  # Example position
camera_direction = torch.tensor([-1, 0, 0], dtype=torch.float16, device=device)  # Looking towards 

initial_radiance = torch.ones_like(initial_wavefront_positions[:, 0])  # Assume the initial radiance is 1
radiance_list = [initial_radiance]

for i in range(1, len(wavefront_positions_list)):
    new_radiance = update_radiance(
        wavefront_positions_list[i],
        wavefront_directions_list[i],
        radiance_list[-1],
        absorb_coeff,
        scatter_coeff,
        test_delta_t,
        camera_direction
    )
    radiance_list.append(new_radiance)

In [ ]:

def accumulate_to_grid_vectorized(radiometric_list, pos_list, grid_shape, absorb_coeff, scatter_coeff):
    # Create a grid to store the accumulated irradiance
    radiometric_grid = torch.zeros(grid_shape, dtype=torch.float16, device=device)

    # Concatenate the positions and irradiances of all steps
    all_positions = torch.cat(pos_list, dim=0).long()
    all_irradiances = torch.cat(radiometric_list, dim=0)

    # Filter out the positions that are outside the grid boundaries
    valid_mask = (all_positions[:, 0] < grid_shape[0]) & (all_positions[:, 0] >= 0) & \
                 (all_positions[:, 1] < grid_shape[1]) & (all_positions[:, 1] >= 0) & \
                 (all_positions[:, 2] < grid_shape[2]) & (all_positions[:, 2] >= 0)

    # Additional check for non-zero absorption or scattering coefficients
    absorb_values = absorb_coeff[all_positions[:, 0], all_positions[:, 1], all_positions[:, 2]]
    scatter_values = scatter_coeff[all_positions[:, 0], all_positions[:, 1], all_positions[:, 2]]
    coeff_mask = (absorb_values > 0) | (scatter_values > 0)

    # Combine valid position mask with coefficients mask
    combined_mask = valid_mask & coeff_mask

    valid_positions = all_positions[combined_mask]
    valid_irradiances = all_irradiances[combined_mask]

    # Accumulate the irradiance values to the grid
    indices = valid_positions[:, 0] * grid_shape[1] * grid_shape[2] + valid_positions[:, 1] * grid_shape[2] + valid_positions[:, 2]
    radiometric_grid.put_(indices, valid_irradiances, accumulate=True)

    return radiometric_grid

# Visualize irradiance grid in 3D projection
def show_radiometric_grid_3d(radiometric_grid, threshold=0.1):
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_box_aspect([1, 1, 1])
    ax.set_xlim(0, radiometric_grid.size(0))
    ax.set_ylim(0, radiometric_grid.size(1))
    ax.set_zlim(0, radiometric_grid.size(2))
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    ax.set_title('Radiometric Grid in 3D Projection')
    radiometric_grid_np = radiometric_grid.cpu().numpy()
    x_points, y_points, z_points = np.where(radiometric_grid_np > threshold)    
    # Create a colormap
    norm = Normalize(vmin=np.min(radiometric_grid_np), vmax=np.max(radiometric_grid_np))
    mappable = ScalarMappable(norm=norm, cmap='hot')
    mappable.set_array(radiometric_grid_np)
    _scatter = ax.scatter(x_points, y_points, z_points, c=radiometric_grid_np[x_points, y_points, z_points], cmap='hot', alpha=0.06)    
    fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=5)        # Add colorbar
    plt.show()


grid_shape = (num_x, num_y, num_z)
cur_radiometric_grid = accumulate_to_grid_vectorized(irradiance_list, wavefront_positions_list, grid_shape, absorb_coeff, scatter_coeff)
show_radiometric_grid_3d(cur_radiometric_grid, threshold=5)